In [2]:
import os
import random
import re
import time
import requests
from lxml import etree
import math
from selenium import webdriver

In [13]:
class Crawler():
    def __init__(self,data_fold,info_url):
        self.data_fold = data_fold
        self.headers = {
            'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36',
            'Referer': 'http://www.dianping.com/beijing/ch50/g183',
            'Cookie':''
        }

        self.info_url = info_url
        self.store_id = self.info_url.split('/')[-1]
        self.fold_path = self.data_fold+'/'+self.store_id
        if not os.path.exists(self.fold_path):
            os.makedirs(self.fold_path)

    def get_pages_1(self,url):
        # 第一种获取页面内容方法--response
        response = requests.get(url = url,headers=self.headers).text
        return response


    def get_pages_2(self):
        '''
        获取店铺评论总数量（两种页面格式，对应两种方法）
        '''
        # 第二种获取页面内容方法--直接下载页面
        driver = webdriver.Chrome('/Users/bu/PycharmProject/MT比赛/DianpingCrawler/chromedriver')
        driver.get(self.info_url)
        time.sleep(30)
        # 暂停30s，在这个时间进行登陆操作

        url = self.info_url+'/review_more/p%s'
        # 获取店铺ID
        # 第一种解析页面方法
        try:
            # 美食店面格式
            driver.get(self.info_url)
            time.sleep(random.randint(5,10))
            response = driver.page_source
            tree = etree.HTML(response)
            assess_number = tree.xpath('//*[@id="defaultcomment-wrapper"]/a/span/text()')[0][1:-1]
        except:
            # 医美店面格式
            self.get_pages_1(url=self.info_url)
            tree = etree.HTML(response)
            d = {}
            d['index'] = self.info_url.split('/')[-1]
            d['name'] = tree.xpath('//*[@id="basic-info"]/h1/text()')[0].replace(' ','').replace('\n','')
            d['locality_region'] = tree.xpath('//*[@id="basic-info"]/div[2]/a/span/text()')[0]
            d['street_address'] = tree.xpath('//*[@id="basic-info"]/div[2]/span[2]/@title')[0]
            try:
                assess_number = tree.xpath('//*[@id="comment"]/h2/a/span/text()')[0][1:-1]
            except:
                assess_number =0
        pages = math.ceil(int(assess_number)/15)
        # 获取评论页数
        if pages==0:
            driver.close()
            return 0
        for page in range(1,pages+1):
            driver.get(url%str(page))
            time.sleep(random.randint(5,10))
            time.sleep(2)
            # 这里把渲染后的网页保存为文件，就不用进行多次爬虫
            html_page = driver.page_source
            with open(self.fold_path+'/page%s.html'%str(page),'w',encoding='utf-8') as f:
                f.write(html_page)
        driver.close()
        return 1

    def get_dictionary(self):
        with open(self.fold_path+'/page1.html','r') as f:
            html_page = f.read()
        # 第二种解析页面方法
        css_url = re.findall('<link rel="stylesheet" type="text/css" href="(//s3plus.meituan.*?)">', html_page)
        css_url = 'http:{}'.format(css_url[0])
        css_content = requests.get(css_url).text
        with open(self.fold_path+'/css.css', 'w', encoding='utf-8') as f:  # 保存css文件
            f.write(css_content)

        # 从中得到字体
        font = re.findall('background-image: url\((.*?)\);', css_content, re.S)  # 提取链接
        maxnum = 0

        for i in font:
            font_url = 'http:{}'.format(i)
            temp_font_content = requests.get(font_url).text
            if len(temp_font_content) > maxnum:
                font_content = temp_font_content
                maxnum = len(font_content)
        with open(self.fold_path+'/font.svg', 'w', encoding='utf-8') as f:  # 保存文件
            f.write(font_content)
        return

    def get_reply(self,html_page,font_content,css_content):
        # 变长
        y_dic = re.findall('<path id=("(\d+)") d="M0 (\d+) H600"/>',font_content)
        y_dic = [(int(idx),int(y)) for non,idx,y in y_dic]
        strategy =1
        if len(y_dic)==0:
            y_dic_2 = re.findall('<text x="0" y="(\d+)">.*?</text>',font_content)
            y_dic_2 = [int(y) for y in y_dic_2]
            strategy = 2
        inf = re.findall('<div class="review-words Hide">(.*?)<div class="less-words">', html_page, re.S)
        inf = inf + re.findall('<div class="review-words">(.*?)</div>',html_page, re.S)

        replylist=[]
        for record in inf:
            inf_copy = record
            svgmti = re.findall('<svgmtsi class="(.*?)">', record)
        # 对于每一个标签
            for class_name in svgmti:
                XY = re.findall('.%s{background:-(.*?)px -(.*?)px;}' % class_name, css_content, re.S)
                #获得坐标
                X = int(float(XY[0][0]) / 14)
                # X的转换两种策略都相同
                if strategy ==1:
                    for i in y_dic:
                        # print(XY[0][1])
                        # print(i)
                        if i[1]> int(float(XY[0][1])):
                            Y = i[0]
                            break
                    fo = re.findall('<textPath xlink:href="#%s" textLength=".*?">(.*?)</textPath>'%str(Y), font_content)
                    # print(fo)
                    inf_copy = re.sub(f'<svgmtsi class="{class_name}"></svgmtsi>', fo[0][X], inf_copy, count=0)
                    # 每得到一个字就把标签换掉
                else:
                    Y = int(float(XY[0][1]))
                    for i in y_dic_2:
                        if i>int(float(Y)):
                            Y=i
                            break
                    fo = re.findall('<text x="0" y="%s">(.*?)</text>'%str(Y), font_content)
                    inf_copy = re.sub(f'<svgmtsi class="{class_name}"></svgmtsi>', fo[0][X], inf_copy, count=0)

            # 删除干扰字符
            inf_copy = re.sub('<img .*? alt="">', '', inf_copy, count=0)
            inf_copy = re.sub('</div>.*?<div class="review-words', '', inf_copy, count=0, flags=re.S)
            inf_copy = inf_copy.replace('Hide">', '')
            inf_copy = inf_copy.replace('">', '')
            inf_copy = inf_copy.replace('\n', '')
            inf_copy = inf_copy.strip()
            replylist.append(inf_copy)
        return replylist


    def run(self):
        flag = self.get_pages_2()
        if flag == 1:
            self.get_dictionary()
            with open(self.fold_path+'/font.svg','r') as f:
                font_content = f.read()
            with open(self.fold_path + '/css.css','r') as f:
                css_content = f.read()
            htmllist = os.listdir(self.fold_path)
            replylist = []
            for html in htmllist:
                if 'page' in html:
                    with open(self.fold_path+'/'+html,'r') as f:
                        html_page = f.read()
                replylist = replylist + self.get_reply(html_page,font_content,css_content)
            return replylist
        else:
            return '无评论'

In [14]:
c = Crawler('Datafold','http://www.dianping.com/shop/G1PvORNW4bl8Kago')
reply_list = c.run()

In [16]:
c = Crawler('Datafold','http://www.dianping.com/shop/H1Ef47MJBEq4bbWj')
reply_list = c.run()

In [17]:
reply_list

['第一次来做医美项目，我是朋友打过觉得不错推荐我去的，环境装修感觉很轻奢，一进门让人感觉很舒服，不由得放松起来。而且他们家真的很专业，都是有专业的医生认真的进行评估测试面诊之后，根据每个人的皮肤状态搭配水光，我选择的是胶原水光，做水光之前先敷麻药，在打的时候一点都不疼，医生也很专业，打的又快又好，手法特别的娴熟。打完水光之后，医生马上给我进行了消毒，然后敷了面膜。医生也跟我讲了一些注意事项，接下来一周内的护肤都是关键。期待早点变成水光肌~',
 '最近脸有点泛红 红血丝 提前预约了舒敏之星 店面很明亮 三层楼 分区域很细化 我就是在三楼的皮肤科 等候区的沙发很舒服 还有糖果零食水先是做了一个特别高清画质的皮肤检测 医生根据图像分析反馈了一些很中肯的小问题 舒敏之星的仪器 感觉在脸上很凉爽 天气马上暖和了 很适合做 会提升小脸脸的屏障修护 之后敷了面膜 医生建议做完可以常敷面膜 还讲了一些注意 整个过程也没有推销 体验感很赞～',
 '三楼皮肤科。 环境舒适  登记测温后开始进行面诊！ 面诊过程中医生告知皮肤出现问题：皮肤缺水 有点小敏感红血丝 注意防晒！ 选择了 BTL紧致提升[感受]体验中感受：    开始在后背上贴了一个带电线的小热帖 。配合脸部仪器操作效果更好！图4 是涂上凝胶后 感觉冰冰凉凉！配合仪器的操作感受不到不舒适，反而一股暖暖的在脸上按摩！  半边脸操作结束后有明显的提升！[效果]体验后效果：脸部提升线条明显，特别是苹果肌与下颌线！敷上面膜后效果更是没得说！',
 '[商户]商户环境/服务：环境非常好 院内干净整洁美观 前台休息区有饮品和糖果 工作人员也非常热情耐心温柔[产品]使用药品/仪器：这次做的光子嫩肤m22 仪器正规 [感受]体验中感受：医生温柔细心仔细 先做了皮肤测试 告诉我所需注意的皮肤问题 也根据我自身问题做了光子嫩肤 而且我比较怕疼 医生还给我了压力球让我捏在手里感觉一点也不疼了 哈哈哈 做完后还耐心的跟我说了我想做的整形项目 心中对整形的疑虑[效果]体验后效果：本来皮肤泛红有痘印  现在感觉提亮了不少泛红痘印也有所减轻 效果感觉非常明显',
 '第一次过来打水光针。是朋友推荐我过来的，因为我看到她的皮肤很好，水润饱满。得知是打了水光针以后的效果，所以我也想来。医生会先帮做一个脸前后的拍照，然后才开始操作的，过程没有任何的疼痛感。打

In [3]:
c = Crawler('Datafold','http://www.dianping.com/shop/k67wZzWBOaEDUtTZ')
with open(c.fold_path+'/font.svg','r') as f:
    font_content = f.read()
with open(c.fold_path + '/css.css','r') as f:
    css_content = f.read()
htmllist = os.listdir(c.fold_path)
replylist = []
for html in htmllist:
    if 'page' in html:
        with open(c.fold_path+'/'+html,'r') as f:
            html_page = f.read()
        replylist = replylist + c.get_reply(html_page,font_content,css_content)

In [6]:
replylist

['很喜欢这个图案，清新自然这个图案比较复杂，纹身师很有耐心，技术也超棒，虽然看着很年轻，但是有8年的纹身经验，我很怕疼，在纹身之前我做了很充分的心理建设，但体验下来感觉这种痛感是完全可以接受的。纹出来的效果也超级棒，非常喜欢，真的是非常靠谱的纹身师。',
 '选了好多店才定的这家，纹身师是有十几年纹身经验的，图案创意和手法都很专业，过程中会经常问我疼不疼，挺温柔的，可以忍受，纹身哪有不疼的图案我选的难度比较高的水墨，从水墨笔触的还原度到渐变的渲染，都处理的很好，颜色饱和度也很高，效果很满意',
 '一直想做一个纹身，不知道哪家好，查询的很久，就选了这家，离我家真的非常远，不过地方还是很好找的，客服小姐姐很贴心，告诉我很多，不厌其烦的，态度非常，到店之后接待人员也很温柔，店内装潢，也是不错的，挺喜欢的',
 '再次来体验纹身，喜欢就拥有，纹身师帅气又有耐心，关键是技术特别好，也不咋疼，也告诉了我注意事项，最近不能吃啥，后期怎么洗，最后呈现的效果很酷，很帅。放心来！',
 '抱着试试的心态买了团购到店里了，员工接待很热情，在纹身是啊麟的沟通建议下，纹了一个蝴蝶加虎头的动物组合图案，非常的帅气，供大家参考',
 '看朋友在这家店做的纹身很棒，就点评上了解了一下，加了店长提前约着过来了，店长很专业，按我的想法纹了一个音符与心的结合，很精致，手艺很棒',
 '看到小伙伴纹身就有了纹身的想法，朋友推荐的一家非常不错的纹身店。电话预约了之后，店员很快就和我联系沟通。纹身过程中也一直聊天放松心情，真是痛并快乐着。',
 '身边朋友都是在这家店纹的，环境不错，纹的整个过程还挺顺利，纹身师的技术超级好，人也很随和，纹的特别仔细，效果很完美！',
 '第二次去了，在点评团购的。第一次就觉得纹身师技术很好，提前电话预约，约的还是上次的小姐姐，不得不说，纹身师的技术真的很好',
 '家门口新开的纹身店 位置很好找 生了孩子后想把一家三口的名字纹在身上 跟设计师说了想法 等着设计图 不喜欢的地方可以修改 最终的效果图非常满意 纹身师手法也非常专业！',
 '很有情调的一家纹身刺青店，一进去之后环境不错，在得知我的需求后给我提供了好几个可供选择的设计图，而且最后做出的效果也很让我满意。整个交谈中特别能体会我的感受，想顾客之所想，推荐！',
 '很专业的一家纹身店，店里环境不错，店员也很有礼貌，

In [65]:
c = Crawler('Datafold','http://www.dianping.com/shop/G1PBax3pqvVH9HSR')
with open(c.fold_path+'/font.svg','r') as f:
    font_content = f.read()
with open(c.fold_path + '/css.css','r') as f:
    css_content = f.read()
htmllist = os.listdir(c.fold_path)
replylist = []
for html in htmllist:
    if 'page' in html:
        with open(c.fold_path+'/'+html,'r') as f:
            html_page = f.read()
        replylist = replylist + c.get_reply(html_page,font_content,css_content)

In [67]:
replylist[:10]

['[商户]商户环境/服务：第一次体验光子嫩肤，找了家附近的一家医院，一进五楼整个环境干净整洁。前台小姐姐很温柔的接待了我们.[产品]使用药品/仪器：我们登记完信息就来到了光子嫩肤室，做之前要彻底清洗干净脸部。[感受]体验中感受：做的过程中会有一丝丝刺痛，不过可以随时沟通力度.做完了会敷一片面膜来补水，仔细的交代了之后一周的护理方法。 做完了立刻觉得自己白了点呢！',
 '[商户]商户环境/服务：是永林中西医结合医院的医美科 很正规 让人放心[产品]使用药品/仪器：我本人的皮肤是混合肌 可能换季加上最近熬夜比较多 眼睛出现一点点细纹 打算做个水光针 改善一下[医生]医生操作/审美：小姐姐很温柔 稍微有一点点疼痛 也是在能接受的范围内 心里默念 会变美会变美 很快就过去了[效果]体验后效果：效果还是不错的 感觉有些改善',
 '位置非常好找，在永辉超市的西北角，永林中西医医院的五楼，进门扫流调卡和健康宝，右手边电梯上五楼。小姐姐小哥哥非常专业，态度也非常好，提前约了五点的，去了就可以做，虽然旁边的房间都有人，还算幸运。妈妈说温温润润的，做起来很舒服，全程大概五十分钟左右，小姐姐一直专心导入，也没有推荐附加项目，总体妈妈十分满意',
 '永林5楼医美，脸上有毛孔和痘印，过来做个光子嫩肤感受一下，过来因为前边的顾客加了项目没有做完，等了大概20分钟左右吧，前台小姐姐给准备了水和零食，进了房间以后有消毒液的味道，疫情期间感觉安全感满满，先敷凝胶然后用仪器打了一边，然后补了一次凝胶又用仪器扫了一边，挺细致的，做完以后敷了一片面膜，全程没有不适感。下次可以在约个时间再来做其他美容项目',
 '[产品]使用药品/仪器：团了一个光子嫩肤，想体验一下子！医生先在脸上涂了一种什么药，冰冰凉凉很舒服～[医生]医生操作/审美：医生动作轻柔，特别有耐心，本来特别紧张的我，都放松下来了。[感受]体验中感受：体验过程非常完美，又做了颈部除皱。整体一点痛感都没有！[效果]体验后效果：效果很立竿见影，不过也要坚持。皮肤嫩嫩的，颈部皱纹也好了很多！坚持✊姐妹们！加油哈环境超级好！',
 '最近换季，皮肤容易起皮，比较干燥，尽管有不定时敷补水面膜，依然干燥，听朋友的推荐来这家体验，店面位于永林三级医院，进医院大厅，做直梯上五层就是美容本店了，接待的是个漂亮小姐姐很温柔，因为客人比较多，在客厅多等了一会，